# 01. Prepare input Data 
## split original ECG signal into frames, saving it scaled.

In [ ]:
val scale = 1/200.0
val samplesPerSecond = 360

val WINDOW = 120   // samples per window
val WindowsPerFrame = 30
val STEP = 2
val SAMPLES = 200000

//val inputDataFile = "a02.dat"

val inputDataFile = "105s1.dat"

val samplesPerFrame = WindowsPerFrame * WINDOW
val secondPerFrame = samplesPerFrame / samplesPerSecond

In [ ]:
import java.io._                                                        
import scala.collection.mutable.ListBuffer   //bE CAREFULL With java imports, better before scala.

In [ ]:
// Databricks

//dbutils.fs.mkdirs("/FileStore/fxo/ted")
//dbutils.fs.ls("FileStore")

//val rootDir = "FileStore/fxo/ted/"
//val fileName = "/dbfs/" + rootDir +"a02.dat"

// GCP

val rootDir = "/home/fsainz/data/"
val fileName = rootDir + inputDataFile 

val framesFileName = "frames-"+ inputDataFile.split('.')(0)+"_" + secondPerFrame +"sLE.csv"


In [ ]:
println("Generating frames of " + samplesPerFrame + " samples (" + 
        + secondPerFrame + "s.) long. ")
        println("OUTPUTFILE =" + framesFileName)

In [ ]:


//val rawData = sc.textFile("/FileStore/tables/gqh5mjl51507524821196/a02.dat", numPartitions)
var inFile =  new File(fileName)
var InFileStream = new FileInputStream( inFile )
val input : DataInputStream  = new DataInputStream(InFileStream)

var frames = scala.collection.mutable.ArrayBuffer.empty[Array[Double]]

var pos = 0
var available = 0


val frameBytes =  samplesPerFrame * 2 // read shorts 2 bytes.
val nFrames = inFile.length / frameBytes
println("Input file length =" + inFile.length  )
println("nFrames = " + nFrames)

var dataFrame = Array[Double]()
for (  w  <- 1 to nFrames.toInt ) {
    
   dataFrame = new Array[Double](samplesPerFrame)
   for ( i <- 0 until samplesPerFrame) {
     // read little indiean and scale
    val value = java.lang.Short.reverseBytes(input.readShort() )  // two bytes
    dataFrame( i ) = (value.toDouble * scale) 
    
  }
  frames.append(dataFrame)
   
}
input.close()

In [ ]:
val csvFilePath = rootDir + framesFileName
// FileWriter

val file = new File(csvFilePath)
val bw = new BufferedWriter(new FileWriter(file))
var line = ""
for ( f <- frames) {
  f.foreach( x => line +=  x.toString + "," )
  bw.write(line.substring(0,line.length-1) + "\n")
  line = ""
 
}
bw.close()
println(csvFilePath + " Written")